In [1]:
# Below are all the necessary libraries needed to get for the project
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
import datetime
import sqlalchemy

# This is the database location where all the extracted data will loaded into. I have to define it here to make available all functions
db_location = f'mssql+pyodbc:ServerName//DBName?trusted_connection=tcon,&driver=ODBC Driver 17 for SQL Server'

In [1]:
import pandas as pd

In [2]:
def extract_transform():
    # Extracting from website(s)
    url = 'https://www.worldometers.info/world-population/population-by-country/'   
    country_list = []

    try:
#Extracting from the stock data API
        _stock_price_api = 'Your API Endpoint goes here'
        price = requests.get(_stock_price_api)
        stk_data = price.json()
        df_stk_price = pd.DataFrame.from_records(stk_data)
        df_stk_price.rename(columns ={'$id':'Row_ID'}, inplace=True)
        df_stk_price['Load_TradeDate'] = datetime.datetime.now() #.strftime("%Y-%m-%d %H:%M")

#Extracting from the bond data  API
        bond_price_api = 'Your API Endpoint goes here'
        bond_price = requests.get(bond_price_api)
        bond_data = bond_price.json()
        df_bond_price = pd.DataFrame.from_records(bond_data)
        df_bond_price['Load_TradeDate'] = datetime.datetime.now()

#Extracting from the etf data API
        etfs_price_api = 'Your API Endpoint goes here'
        etfs_price = requests.get(etfs_price_api)
        etfs_data = etfs_price.json()
        df_etfs_price = pd.DataFrame.from_records(etfs_data)
        df_etfs_price['Load_TradeDate'] = datetime.datetime.now()

#Here we fetching data from a website
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'lxml')
        rows = soup.find('table', {'id':'example2'}).find('tbody').find_all('tr')
        for row in rows:
            dic = {}
            dic['Country_Key'] = row.find_all('td')[1].text
            dic['Country'] = row.find_all('td')[1].text
            dic['Population(2020)'] = row.find_all('td')[2].text
            dic['Yearly Cahnge'] = row.find_all('td')[3].text
            dic['Net Change'] = row.find_all('td')[4].text
            dic['Density(P/Km^2)'] = row.find_all('td')[5].text
            dic['Land Area(Km^2)'] = row.find_all('td')[6].text
            dic['Migrants(Net)'] = row.find_all('td')[7].text
            dic['Fert.Rate'] = row.find_all('td')[8].text
            dic['Med Age'] = row.find_all('td')[9].text
            dic['Urban Pop%'] = row.find_all('td')[10].text
            dic['World Share'] = row.find_all('td')[11].text
            dic['Load_Txn_Date'] = datetime.datetime.now()

            country_list.append(dic)
        country_df = pd.DataFrame(country_list)
        country_df['Country_Key'] = country_df['Country_Key'].str.replace(' ','')
    except Exception as error:
        print(error)

#This function extracts the required values from a row with dictionary/list/tuple
    def debt_dict(row):
        r = ''
        if isinstance(row, dict):
            convert_dict = list(row.values())
            r = convert_dict[0]
            return r
        elif isinstance(row, list):
            return row[0]
        elif isinstance(row, tuple):
            return row[0]
        else:
            return row
    try:       
        pd_csv_local_file = pd.read_csv('employment_data.csv')
        pd_csv_local_file['TxnDate'] = datetime.datetime.now()
    
        pd_xml_local_file = pd.read_xml('salary_data.xml')
        pd_xml_local_file['TxnDate'] = datetime.datetime.now()
    
        pd_json_local_file = pd.read_json('user_cc_data.json')
        pd_json_local_file['Load_TxnDate'] = datetime.datetime.now()
        # There is a field that some of it rows contains a mix of dic, list, and tuple.
        # So we will use the function (debt_dict) created above to uniquely make the corrections per row
        pd_json_local_file["debt"] = pd_json_local_file["debt"].apply(debt_dict)
    except Exception as error:
        print(error)
    

    return df_stk_price, country_df, pd_csv_local_file, pd_xml_local_file, pd_json_local_file 

def load():
    try:
#Creating a connection to SQL Server
        sql_server_conn = sqlalchemy.create_engine(db_location)
#Loading data to DB; it creates the table(s) if not exists! 
        extract_transform()[0].to_sql("nse_stk_price", con = sql_server_conn, if_exists= "append", index=False)
        extract_transform()[1].to_sql("world_population", con = sql_server_conn, if_exists= "replace", index=False)
        extract_transform()[2].to_sql("local_csv", con = sql_server_conn, if_exists= "append", index=False)
        extract_transform()[3].to_sql("local_xml", con = sql_server_conn, if_exists= "append", index=False)
        extract_transform()[4].to_sql("local_json", con = sql_server_conn, if_exists= "append", index=False)

    except Exception as error:
        print(error)

load()